# 第 8 章 异方差性 
## C8.1【异方差模型】
考虑如下解释睡眠行为的模型： 
$$sleep=\beta_0+\beta_1 totwrk+\beta_2 educ +\beta_3 age+\beta_4 age^2+\beta_5 yngkid+\beta_6 male+\mu$$
（1）写出一个模型，容许u的方差在男女之间有所不同。这个方差不应该取决于其他因素。 

$$Var(\mu|totwrk,educ,age,yngkid,male)=Var(\mu|male)=\delta_0+\delta_1 male$$

（2）利用SLEEP75.RAW 中的数据估计异方差模型中的参数。（你必须先用OLS估计sleep方程，以得到OLS残差。）u的估计方差对于男人和女人而言哪个更高？ 

（3）u 的方差是否对于男女而言有显著不同？ 

In [1]:
library(wooldridge)
library(haven)

In [2]:
SLEEP75 <- read_dta("C:/Users/bradl/SRM/data/SLEEP75.DTA")
attach(SLEEP75)

The following object is masked from package:datasets:

    sleep



In [3]:
fit1<-lm(sleep~totwrk+educ+age+I(age^2)+yngkid+male) #拟合上述模型 
summary(fit1) #


Call:
lm(formula = sleep ~ totwrk + educ + age + I(age^2) + yngkid + 
    male)

Residuals:
     Min       1Q   Median       3Q      Max 
-2378.00  -243.29     6.73   259.24  1350.19 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept) 3840.85207  239.41393  16.043   <2e-16 ***
totwrk        -0.16342    0.01816  -8.997   <2e-16 ***
educ         -11.71327    5.87195  -1.995   0.0465 *  
age           -8.69740   11.32909  -0.768   0.4429    
I(age^2)       0.12844    0.13467   0.954   0.3405    
yngkid        -0.02280   50.27641   0.000   0.9996    
male          87.75455   34.66794   2.531   0.0116 *  
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 418 on 699 degrees of freedom
Multiple R-squared:  0.1228,	Adjusted R-squared:  0.1152 
F-statistic:  16.3 on 6 and 699 DF,  p-value: < 2.2e-16


In [4]:
a<-residuals(fit1) #将fit1 的残差保存在a 中 
fit2<-lm(I(a^2)~male) #将残差平方对 male 回归 
summary(fit2) 


Call:
lm(formula = I(a^2) ~ male)

Residuals:
    Min      1Q  Median      3Q     Max 
-189359 -155559 -111662   26265 5465531 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)   189359      20546   9.216   <2e-16 ***
male          -28850      27297  -1.057    0.291    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 359400 on 704 degrees of freedom
Multiple R-squared:  0.001584,	Adjusted R-squared:  0.000166 
F-statistic: 1.117 on 1 and 704 DF,  p-value: 0.2909


In [5]:
detach(SLEEP75) 

## C8.2【异方差-稳健标准误；用 log()消除异方差】 
（1）利用HPRICE1.RAW 中的数据得到方程（8.17）的异方差-稳健标准误。讨论其与通常的标准误之间是否存在任何重要差异。 
$$ price=\beta_0+\beta_1 lotsize+\beta_2 sqrft+\beta_3 bdrms+\mu \tag{8.17}$$ 
（2）对方程8.18 重复第（1）步操作。 

（3）此例对异方差性和对因变量所做的变换说明了什么

In [6]:
attach(hprice1) 

In [7]:
fit1<-lm(price~lotsize+sqrft+bdrms) 
summary(fit1) 


Call:
lm(formula = price ~ lotsize + sqrft + bdrms)

Residuals:
     Min       1Q   Median       3Q      Max 
-120.026  -38.530   -6.555   32.323  209.376 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept) -2.177e+01  2.948e+01  -0.739  0.46221    
lotsize      2.068e-03  6.421e-04   3.220  0.00182 ** 
sqrft        1.228e-01  1.324e-02   9.275 1.66e-14 ***
bdrms        1.385e+01  9.010e+00   1.537  0.12795    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 59.83 on 84 degrees of freedom
Multiple R-squared:  0.6724,	Adjusted R-squared:  0.6607 
F-statistic: 57.46 on 3 and 84 DF,  p-value: < 2.2e-16


In [10]:
library(lmtest) #载入求异方差-稳健标准误需要的包 
library(car) #载入求异方差-稳健标准误需要的包 
library(zoo)
coeftest(fit1, vcov=hccm(fit1,type = "hc0")) 


t test of coefficients:

               Estimate  Std. Error t value  Pr(>|t|)    
(Intercept) -21.7703081  36.2843444 -0.6000   0.55013    
lotsize       0.0020677   0.0012227  1.6912   0.09451 .  
sqrft         0.1227782   0.0173178  7.0897 3.883e-10 ***
bdrms        13.8525217   8.2836880  1.6723   0.09819 .  
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1


In [12]:
fit2<-lm(log(price)~log(lotsize)+log(sqrft)+bdrms) #拟合上述模型 
summary(fit2)


Call:
lm(formula = log(price) ~ log(lotsize) + log(sqrft) + bdrms)

Residuals:
     Min       1Q   Median       3Q      Max 
-0.68422 -0.09178 -0.01584  0.11213  0.66899 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept)  -1.29704    0.65128  -1.992   0.0497 *  
log(lotsize)  0.16797    0.03828   4.388 3.31e-05 ***
log(sqrft)    0.70023    0.09287   7.540 5.01e-11 ***
bdrms         0.03696    0.02753   1.342   0.1831    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 0.1846 on 84 degrees of freedom
Multiple R-squared:  0.643,	Adjusted R-squared:  0.6302 
F-statistic: 50.42 on 3 and 84 DF,  p-value: < 2.2e-16


In [13]:
coeftest(fit2, vcov=hccm(fit2,type = "hc0"))


t test of coefficients:

              Estimate Std. Error t value  Pr(>|t|)    
(Intercept)  -1.297042   0.763351 -1.6991   0.09299 .  
log(lotsize)  0.167967   0.040520  4.1453 8.067e-05 ***
log(sqrft)    0.700232   0.101442  6.9028 9.014e-10 ***
bdrms         0.036958   0.029898  1.2362   0.21984    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1


可以看到在采用对数形式的变量后，普通标准误和异方差-稳健标准误差别不大，而且 log(lotsize)和log(sqrft)在普通标准误和异方差-稳健标准误中都十分显著。 

（3）此例对异方差性和对因变量所做的变换说明了什么？ 

说明采用对数形式的变量有助于控制异方差。

## C8.3【怀特异方差检验；布罗施-帕甘检验（BP test）】 

在方程（8.18）中应用异方差性的完全怀特检验[参见方程（8.19）]。利用$\chi^2$形式的统计量并计算p值。你得到什么结论？ 
$$log(price)=\beta_0+\beta_1log (lotsize)+\beta_2log(sqrft)+\beta_3 bdrms+\mu  \tag{8.19}$$ 

In [14]:
attach(hprice1)  

The following objects are masked from hprice1 (pos = 7):

    assess, bdrms, colonial, lassess, llotsize, lotsize, lprice,
    lsqrft, price, sqrft



In [15]:
fit1<-lm(log(price)~log(lotsize)+log(sqrft)+bdrms)
library(lmtest)
bptest(fit1,~(log(lotsize)+log(sqrft)+bdrms)^2+I(log(lotsize)^2)+I(log(sqrft)^2)+I(bdrms^2)) 


	studentized Breusch-Pagan test

data:  fit1
BP = 9.5495, df = 9, p-value = 0.3882


p 值为0.3882，不能支持有异方差的存在

## C8.4【布罗施-帕甘检验（BP test）；特殊怀特检验】 

本题使用VOTE1.RAW 中的数据。 

（1）估计一个以voteA 为因变量并以prtystrA、democA、log(expendA)和 log(expendB)为自变量的模型。得到OLS残差$\hat{\mu_i}$，并将这些残差对所有的自变量进行回归。解释你为什么得到$R^2=0$。 

（ 2）现在计算异方差性的布罗施-帕甘检验。使用F 统计量的形式并报告p 值。 

（3）同样利用F统计量形式计算异方差性的特殊怀特检验。现在异方差性的证据有多强?

In [17]:
VOTE1 <- read_dta("C:/Users/bradl/SRM/data/VOTE1.DTA")
attach(VOTE1) 

In [18]:
fit1<-lm(voteA~ prtystrA+democA+log(expendA)+log(expendB)) #拟合上述模型 
summary(fit1) #


Call:
lm(formula = voteA ~ prtystrA + democA + log(expendA) + log(expendB))

Residuals:
    Min      1Q  Median      3Q     Max 
-18.576  -4.864  -1.146   4.903  24.566 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept)  37.66141    4.73604   7.952 2.56e-13 ***
prtystrA      0.25192    0.07129   3.534  0.00053 ***
democA        3.79294    1.40652   2.697  0.00772 ** 
log(expendA)  5.77929    0.39182  14.750  < 2e-16 ***
log(expendB) -6.23784    0.39746 -15.694  < 2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 7.573 on 168 degrees of freedom
Multiple R-squared:  0.8012,	Adjusted R-squared:  0.7964 
F-statistic: 169.2 on 4 and 168 DF,  p-value: < 2.2e-16


In [20]:
a<-residuals(fit1) #将残差保存在a 中 
fit2<-lm(a~prtystrA+democA+log(expendA)+log(expendB)) 
summary(fit2) 


Call:
lm(formula = a ~ prtystrA + democA + log(expendA) + log(expendB))

Residuals:
    Min      1Q  Median      3Q     Max 
-18.576  -4.864  -1.146   4.903  24.566 

Coefficients:
               Estimate Std. Error t value Pr(>|t|)
(Intercept)  -2.705e-15  4.736e+00       0        1
prtystrA      1.563e-16  7.129e-02       0        1
democA       -4.577e-16  1.407e+00       0        1
log(expendA) -3.685e-16  3.918e-01       0        1
log(expendB) -5.594e-16  3.975e-01       0        1

Residual standard error: 7.573 on 168 degrees of freedom
Multiple R-squared:  5.211e-32,	Adjusted R-squared:  -0.02381 
F-statistic: 2.189e-30 on 4 and 168 DF,  p-value: 1


In [21]:
library(lmtest) #载入做异方差检验所需的包 
bptest(fit1) 


	studentized Breusch-Pagan test

data:  fit1
BP = 9.0934, df = 4, p-value = 0.05881


In [23]:
bptest(fit1,~fitted(fit1)+I(fitted(fit1)^2)) 
qf(1-0.05881,2,170) 
detach(VOTE1)


	studentized Breusch-Pagan test

data:  fit1
BP = 5.49, df = 2, p-value = 0.06425


[1] 2.881198

## C8.5【只对一个截距项回归；条件计数】 
本题使用PNTSPRD.RAW 中的数据。 

（1）变量sprdcvr 是一个二值变量，若在大学篮球比赛中实际分数差距超过拉斯维加斯让分，则此变量取值1。sprdcvr 的期望（比方说$\mu$）表示在一场随机抽取的比赛中分差超过让分的概率。在10%的显著性水平上相对于$H_1：\mu\neq 0.5$检验$H_0:\mu=0.5$，并讨论你的结果。（提示：将sprdcvr只对一个截距项进行回归遍得到一个t 统计量，利用这个t统计量很容易完成。） 

（2）553 个样本中有多少场比赛是在中立场地进行的？ 

（3）估计线性概率模型  $$sprdcvr=\beta_0+\beta_1 favhome+\beta_2 neutral +\beta_3 fav25 +\beta_4 und25+\mu$$ 并以通常的形式报告结论。（报告通常的标准误和异方差——稳健的标准误。）哪个变量在实际上和统计上都是显著的？

（4）解释为什么在虚拟假设$H_0：\beta_1=\beta_2=\beta_3=\beta_4=0$下，模型中不存在异方差性。 

（5）利用通常的F统计量检验第（4）部分的虚拟假设，你得到什么结论？ 

（6）给定上述分析，你会不会认为，利用赛前可利用的信息，有可能系统地预测拉斯维加斯 让分能否实现？

In [25]:
PNTSPRD <- read_dta("C:/Users/bradl/SRM/data/PNTSPRD.DTA")
attach(PNTSPRD) 

In [26]:
fit1<-lm(sprdcvr~1) 
summary(fit1) 


Call:
lm(formula = sprdcvr ~ 1)

Residuals:
    Min      1Q  Median      3Q     Max 
-0.5154 -0.5154  0.4846  0.4846  0.4846 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)  0.51537    0.02127   24.23   <2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 0.5002 on 552 degrees of freedom


In [27]:
sum(neutral==1) 

[1] 35

In [28]:
fit2<-lm(sprdcvr~favhome+neutral+fav25+und25) 
summary(fit2)


Call:
lm(formula = sprdcvr ~ favhome + neutral + fav25 + und25)

Residuals:
    Min      1Q  Median      3Q     Max 
-0.6072 -0.5242  0.3928  0.4758  0.5339 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)  0.48957    0.04476  10.938   <2e-16 ***
favhome      0.03459    0.04972   0.696    0.487    
neutral      0.11762    0.09466   1.242    0.215    
fav25       -0.02347    0.05019  -0.468    0.640    
und25        0.01787    0.09188   0.195    0.846    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 0.5012 on 548 degrees of freedom
Multiple R-squared:  0.0034,	Adjusted R-squared:  -0.003874 
F-statistic: 0.4674 on 4 and 548 DF,  p-value: 0.7597


In [29]:
library(car) #载入car 包（首次使用用install.packages("car")安装。） 
myH0 <- c("favhome","neutral","fav25","und25") #用联合检验所涉及的变量组成向量，构成H0。 
linearHypothesis(fit2,myH0) 

Res.Df,RSS,Df,Sum of Sq,F,Pr(>F)
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
552,138.1193,NA,NA,NA,NA
548,137.6497,4,0.4696334,0.4674167,0.7596745


In [30]:
detach(PNTSPRD) 

## C8.6【加权最小二乘法（线性概率模型）；F 检验】 

在例7.12 中，我们估计了一个线性概率模型以说明一个年轻人在1986 年是否被拘捕
$$arr86=\beta_0+\beta_1 pcnv+\beta_2 avgsen+\beta_3 tottime +\beta_4 ptime86+\beta_5 qemp86+\mu$$
（1）用OLS 估计此模型，并验证其全部估计值都严格地介于0 和1 之间。最大和最小的估计 值各是多少？ 

（2）像8.5 节讨论的那样，用加权最小二乘法估计这个方程。 

（3）用WLS 估计值决定avgsen 和tottime 在5%的显著性水平上是否联合显著。 


In [32]:
CRIME1 <- read_dta("C:/Users/bradl/SRM/data/CRIME1.DTA")
attach(CRIME1) 

In [37]:
CRIME1<-within(CRIME1,{       
    arr86 <- NA       
    arr86[narr86>0] <- 1       
    arr86[narr86==0] <- 0 })   
fit1<-lm(arr86~pcnv+avgsen+tottime+ptime86+qemp86, data = CRIME1) #拟合上述模型 
summary(fit1) 
max(fitted(fit1)) 
min(fitted(fit1)) 


Call:
lm(formula = arr86 ~ pcnv + avgsen + tottime + ptime86 + qemp86, 
    data = CRIME1)

Residuals:
    Min      1Q  Median      3Q     Max 
-0.5577 -0.2889 -0.2157  0.5734  0.8931 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept)  0.440615   0.017233  25.568  < 2e-16 ***
pcnv        -0.162445   0.021237  -7.649 2.79e-14 ***
avgsen       0.006113   0.006452   0.947    0.344    
tottime     -0.002262   0.004978  -0.454    0.650    
ptime86     -0.021966   0.004635  -4.739 2.25e-06 ***
qemp86      -0.042829   0.005405  -7.925 3.31e-15 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 0.4373 on 2719 degrees of freedom
Multiple R-squared:  0.04735,	Adjusted R-squared:  0.0456 
F-statistic: 27.03 on 5 and 2719 DF,  p-value: < 2.2e-16


[1] 0.5576897

[1] 0.006643125

In [34]:
yhat<-fitted(fit1) 
hhat<-yhat*(1-yhat)
fit2<-lm(arr86~pcnv+avgsen+tottime+ptime86+qemp86, weight=1/hhat, data = CRIME1)
summary(fit2)


Call:
lm(formula = arr86 ~ pcnv + avgsen + tottime + ptime86 + qemp86, 
    data = CRIME1, weights = 1/hhat)

Weighted Residuals:
    Min      1Q  Median      3Q     Max 
-1.1219 -0.6288 -0.5141  1.1537  2.9166 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept)  0.447597   0.017992  24.877  < 2e-16 ***
pcnv        -0.167844   0.018912  -8.875  < 2e-16 ***
avgsen       0.005367   0.005115   1.049    0.294    
tottime     -0.001761   0.003251  -0.542    0.588    
ptime86     -0.024619   0.003045  -8.085 9.28e-16 ***
qemp86      -0.045188   0.005423  -8.333  < 2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 0.9909 on 2719 degrees of freedom
Multiple R-squared:  0.07439,	Adjusted R-squared:  0.07268 
F-statistic:  43.7 on 5 and 2719 DF,  p-value: < 2.2e-16


In [36]:
myH0 <- c("avgsen","tottime") #用联合检验所涉及的变量组成向量，构成H0。 
linearHypothesis(fit2,myH0) 
detach(CRIME1) 

Res.Df,RSS,Df,Sum of Sq,F,Pr(>F)
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2721,2671.582,NA,NA,NA,NA
2719,2669.845,2,1.737882,0.8849395,0.412858


## C8.7【异方差-稳健标准误；值域；条件计数】 

本题利用LOANAPP.RAW 中的数据。 

（1）估计计算机习题C7.8 第（3）部分中的方程，计算其异方差-稳健的标准误。将$\beta_{white}$的95% 的置信区间与非稳健的置信区间相比较。 

（2）由第（1）部分的回归拟计算拟合值。其中有没有哪个估计值小于 0？有没有哪个估计值 大于1？而这些情况对加权最小二乘估计的应用意味着什么？ 

In [38]:
attach(loanapp) 

The following object is masked from hprice1 (pos = 3):

    price

The following object is masked from hprice1 (pos = 8):

    price



In [39]:
fit1<-lm(approve~white+hrat+obrat+loanprc+unem+male+married+dep+sch+cosign+chist+pubrec+mortlat1+mortlat2+vr) #拟合上述模型，采用非异方差-稳健标准误 
summary(fit1)


Call:
lm(formula = approve ~ white + hrat + obrat + loanprc + unem + 
    male + married + dep + sch + cosign + chist + pubrec + mortlat1 + 
    mortlat2 + vr)

Residuals:
     Min       1Q   Median       3Q      Max 
-1.06482  0.00781  0.06387  0.13673  0.71105 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept)  0.936731   0.052735  17.763  < 2e-16 ***
white        0.128820   0.019732   6.529 8.44e-11 ***
hrat         0.001833   0.001263   1.451   0.1469    
obrat       -0.005432   0.001102  -4.930 8.92e-07 ***
loanprc     -0.147300   0.037516  -3.926 8.92e-05 ***
unem        -0.007299   0.003198  -2.282   0.0226 *  
male        -0.004144   0.018864  -0.220   0.8261    
married      0.045824   0.016308   2.810   0.0050 ** 
dep         -0.006827   0.006701  -1.019   0.3084    
sch          0.001753   0.016650   0.105   0.9162    
cosign       0.009772   0.041139   0.238   0.8123    
chist        0.133027   0.019263   6.906 6.72e-12 ***
pubrec      -0.241

In [42]:
coeftest(fit1, vcov=hccm(fit1,type = "hc0")) #获得异方差-稳健标准误


t test of coefficients:

              Estimate Std. Error t value  Pr(>|t|)    
(Intercept)  0.9367312  0.0591471 15.8373 < 2.2e-16 ***
white        0.1288196  0.0257641  5.0000 6.246e-07 ***
hrat         0.0018330  0.0014611  1.2546  0.209789    
obrat       -0.0054318  0.0013256 -4.0977 4.343e-05 ***
loanprc     -0.1473001  0.0376812 -3.9091 9.579e-05 ***
unem        -0.0072989  0.0036971 -1.9742  0.048496 *  
male        -0.0041441  0.0192259 -0.2155  0.829361    
married      0.0458241  0.0171672  2.6693  0.007664 ** 
dep         -0.0068274  0.0068757 -0.9930  0.320848    
sch          0.0017525  0.0170763  0.1026  0.918269    
cosign       0.0097722  0.0394215  0.2479  0.804245    
chist        0.1330267  0.0245201  5.4252 6.507e-08 ***
pubrec      -0.2419268  0.0426181 -5.6766 1.579e-08 ***
mortlat1    -0.0572511  0.0659540 -0.8680  0.385476    
mortlat2    -0.1137234  0.0906993 -1.2539  0.210046    
vr          -0.0314408  0.0144265 -2.1794  0.029423 *  
---
Signif. codes:  0 

$\beta_{white}$的异方差-稳健标准误是0.0257641，而非异方差-稳健标准误是 0.019732。异方差-稳健的95%的置信区间是（0.078321960，0.1793172）。非异方差-稳健的95%的置信区间是（0.09014488，0.1674943）。因此非异方差-稳健的 95%的置信区间更窄。 

In [44]:
range(fitted(fit1)) #求拟合值的值域，结果为[0.2273447，1.1729878]，因此没有拟合值小于0， 但是又大于 1 的。 
sum(fitted(fit1)>1) #计算有多少个拟合值大于1，结果为213。23因此除非对这213 个大于1 的拟合值进行调整，否则不能运用加权最小二乘法。 
detach(loanapp) 

[1] 0.2273447 1.1729878

[1] 213

## C8.8【特殊怀特异方差检验；加权最小二乘法（WLS）；异方差-稳健标准误】

本题利用数据集GPA1.RAW。 

（1）利用OLS估计一个将colGPA 与hsGPA，ACT，skipped 和PC 相联系的模型。求OLS残差。 

（2）计算异方差性的怀特检验特殊情形。在$\hat{\mu_i}^2$对$\widehat{colGPA_i}$和$\widehat{colGPA_i}^2$的回归中，求拟合值$\hat{h_i}$。 

（3）验证第（2）部分得到的拟合值都严格为正。然后利用权数$1/\hat{h_i}$求加权最小二乘估计值。根据对应的 OLS 估计值，将逃课和拥有计算机之影响的加权最小二乘估计值与对应OLS 估计 值相比较。他们的统计显著性如何？ 

（4）在第（3）部分的 WLS 估计中，求异方差-稳健的标准误。换言之，容许第（2）部分中 所估计的方差函数可能误设（参见问题8.4）。标准误与第（3）部分相比有很大变化吗？ 

In [46]:
GPA1 <- read_dta("C:/Users/bradl/SRM/data/GPA1.DTA")
attach(GPA1) 

The following objects are masked from package:wooldridge:

    alcohol, campus



In [47]:
fit1<-lm(colGPA~hsGPA+ACT+skipped+PC) #拟合上述模型 
summary(fit1)
a<-residuals(fit1)


Call:
lm(formula = colGPA ~ hsGPA + ACT + skipped + PC)

Residuals:
     Min       1Q   Median       3Q      Max 
-0.84006 -0.20392 -0.03352  0.25346  0.74558 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)  1.35651    0.32750   4.142 6.01e-05 ***
hsGPA        0.41295    0.09243   4.468 1.65e-05 ***
ACT          0.01334    0.01044   1.278  0.20353    
skipped     -0.07103    0.02625  -2.706  0.00768 ** 
PC           0.12444    0.05731   2.171  0.03165 *  
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 0.3251 on 136 degrees of freedom
Multiple R-squared:  0.2593,	Adjusted R-squared:  0.2375 
F-statistic:  11.9 on 4 and 136 DF,  p-value: 2.553e-08


In [48]:
bptest(fit1,~fitted(fit1)+I(fitted(fit1)^2)) #做特殊怀特异方差检验


	studentized Breusch-Pagan test

data:  fit1
BP = 6.9576, df = 2, p-value = 0.03084


In [50]:
fit2<-lm(I(a^2)~ fitted(fit1)+I(fitted(fit1)^2)) 
summary(fit2)
hhat<-fitted(fit2)


Call:
lm(formula = I(a^2) ~ fitted(fit1) + I(fitted(fit1)^2))

Residuals:
     Min       1Q   Median       3Q      Max 
-0.13286 -0.07802 -0.04020  0.04954  0.60632 

Coefficients:
                   Estimate Std. Error t value Pr(>|t|)
(Intercept)       -0.321838   2.005841  -0.160    0.873
fitted(fit1)       0.129600   1.316763   0.098    0.922
I(fitted(fit1)^2)  0.002946   0.215660   0.014    0.989

Residual standard error: 0.1237 on 138 degrees of freedom
Multiple R-squared:  0.04934,	Adjusted R-squared:  0.03557 
F-statistic: 3.581 on 2 and 138 DF,  p-value: 0.03045


In [52]:
range(hhat) #求拟合值的值域，结果为[0.02738136，0.16480429]，满足严格为正。
fit3<-lm(colGPA~hsGPA+ACT+skipped+PC, weight=1/hhat) 
summary(fit3) 

[1] 0.02738136 0.16480429


Call:
lm(formula = colGPA ~ hsGPA + ACT + skipped + PC, weights = 1/hhat)

Weighted Residuals:
    Min      1Q  Median      3Q     Max 
-2.6994 -0.6892 -0.1191  0.7963  2.5098 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept)  1.401564   0.298430   4.696 6.39e-06 ***
hsGPA        0.402506   0.083362   4.828 3.65e-06 ***
ACT          0.013162   0.009827   1.339 0.182698    
skipped     -0.076365   0.022173  -3.444 0.000762 ***
PC           0.126005   0.056339   2.237 0.026945 *  
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 1.013 on 136 degrees of freedom
Multiple R-squared:  0.3062,	Adjusted R-squared:  0.2858 
F-statistic: 15.01 on 4 and 136 DF,  p-value: 3.488e-10


In [54]:
coeftest(fit3, vcov=hccm(fit3,type = "hc0")) 
detach(GPA1) 


t test of coefficients:

             Estimate Std. Error t value  Pr(>|t|)    
(Intercept)  1.401564   0.305039  4.5947 9.782e-06 ***
hsGPA        0.402506   0.084790  4.7471 5.162e-06 ***
ACT          0.013162   0.010228  1.2869 0.2003075    
skipped     -0.076365   0.020830 -3.6662 0.0003521 ***
PC           0.126005   0.057969  2.1737 0.0314608 *  
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1


## C8.9【可行 GLS；可行 GLS 的布罗施-帕甘检验】 
在例8.7 中，我们计算了香烟需求方程的OLS 和一系列WLS 估计值。 

（1）求方程（8.35）中的OLS 估计值。 

（2）求出方程（8.36）的WLS 古籍中所用的$\hat{h_i}$，并重新得到方程（8.36）。根据这个方程，求未加权残差和拟合值；分别称之为$\hat{u_i}$和$\hat{y_i}$。（比如在stata 中，未加权的残差和拟合值默认给出。）

（3）令$\tilde{u_i}=\hat{u_i}/\sqrt{\hat{h_i}}$和$\tilde{y_i}=\hat{y_i}/\sqrt{\hat{h_i}}$表示加权量。通过将$\tilde{u_i}^2$对$\tilde{y_i}$和$\tilde{y_i}^2$回归，进行怀特异方差性检验的特殊情形，一定要照常包含截距项。你在加权残差中发现了异方差性吗？
 
（4）第（3）部分的结论对于求式（8.36）时建议使用的同方差形式有何含义？ （5）在容许方差函数被误设的情况下，求WLS 估计值的确当标准误。 

In [56]:
SMOKE <- read_dta("C:/Users/bradl/SRM/data/SMOKE.DTA")
attach(SMOKE) 

In [57]:
fit1<-lm(cigs~log(income)+log(cigpric)+educ+age+I(age^2)+restaurn) #拟合上述模型 
summary(fit1) #查看拟合结果 
a<-fitted(fit1) #将fit1 的拟合值保存在a中 


Call:
lm(formula = cigs ~ log(income) + log(cigpric) + educ + age + 
    I(age^2) + restaurn)

Residuals:
    Min      1Q  Median      3Q     Max 
-15.819  -9.381  -5.975   7.922  70.221 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept)  -3.639826  24.078659  -0.151  0.87988    
log(income)   0.880268   0.727783   1.210  0.22682    
log(cigpric) -0.750862   5.773342  -0.130  0.89655    
educ         -0.501498   0.167077  -3.002  0.00277 ** 
age           0.770694   0.160122   4.813 1.78e-06 ***
I(age^2)     -0.009023   0.001743  -5.176 2.86e-07 ***
restaurn     -2.825085   1.111794  -2.541  0.01124 *  
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 13.4 on 800 degrees of freedom
Multiple R-squared:  0.05274,	Adjusted R-squared:  0.04563 
F-statistic: 7.423 on 6 and 800 DF,  p-value: 9.499e-08


In [58]:
b<-residuals(fit1) #将fit1 的残差保存在b 中 
fit2<-lm(I(log(b^2))~ log(income)+log(cigpric)+educ+age+I(age^2)+restaurn) #做方程8.32 中的回归 

In [59]:
c<-fitted(fit2) #得到fit2 中的拟合值 
hhat<-exp(c) 
fit3<-lm(cigs~log(income)+log(cigpric)+educ+age+I(age^2)+restaurn, weight=1/hhat) #重新计算加 权最小二乘得到的方程 8.36 
summary(fit3) 


Call:
lm(formula = cigs ~ log(income) + log(cigpric) + educ + age + 
    I(age^2) + restaurn, weights = 1/hhat)

Weighted Residuals:
    Min      1Q  Median      3Q     Max 
-1.9036 -0.9532 -0.8099  0.8415  9.8556 

Coefficients:
               Estimate Std. Error t value Pr(>|t|)    
(Intercept)   5.6354627 17.8031394   0.317 0.751673    
log(income)   1.2952393  0.4370117   2.964 0.003128 ** 
log(cigpric) -2.9403117  4.4601446  -0.659 0.509931    
educ         -0.4634464  0.1201587  -3.857 0.000124 ***
age           0.4819480  0.0968082   4.978 7.86e-07 ***
I(age^2)     -0.0056272  0.0009395  -5.990 3.17e-09 ***
restaurn     -3.4610640  0.7955050  -4.351 1.53e-05 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 1.579 on 800 degrees of freedom
Multiple R-squared:  0.1134,	Adjusted R-squared:  0.1068 
F-statistic: 17.06 on 6 and 800 DF,  p-value: < 2.2e-16


In [61]:
uhat<-residuals(fit3) 
yhat<-fitted(fit3) 
u<-uhat/sqrt(hhat) 
y<-yhat/sqrt(hhat)
fit4<-lm(I(u^2)~y+I(y^2)) 
summary(fit4)
#根据式8.15 计算F值，结果F= 11.15519，在1%水平上显 著。因此是存在异方差的有力证据。 


Call:
lm(formula = I(u^2) ~ y + I(y^2))

Residuals:
   Min     1Q Median     3Q    Max 
-5.579 -1.801 -1.306 -0.855 90.871 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)   
(Intercept) -0.06146    0.96043  -0.064  0.94899   
y            0.28667    1.41212   0.203  0.83918   
I(y^2)       2.40597    0.78615   3.060  0.00228 **
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 6.414 on 804 degrees of freedom
Multiple R-squared:  0.027,	Adjusted R-squared:  0.02458 
F-statistic: 11.15 on 2 and 804 DF,  p-value: 1.667e-05


In [62]:
coeftest(fit3, vcov=hccm(fit3,type = "hc0")) #获得WLS 估计的异方差-稳健标准误，


t test of coefficients:

               Estimate Std. Error t value  Pr(>|t|)    
(Intercept)   5.6354627 37.1611619  0.1516  0.879502    
log(income)   1.2952393  0.5327686  2.4311  0.015270 *  
log(cigpric) -2.9403117  8.9314572 -0.3292  0.742084    
educ         -0.4634464  0.1484143 -3.1227  0.001857 ** 
age           0.4819480  0.1144917  4.2095 2.850e-05 ***
I(age^2)     -0.0056272  0.0011719 -4.8018 1.877e-06 ***
restaurn     -3.4610640  0.7127922 -4.8556 1.443e-06 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1


In [64]:
summary(fit3) #可以看到除了restaurn 之外，所有变量的异方差-稳健标准误更大。 
detach(SMOKE)  #解除绑定数据集SMOKE。 


Call:
lm(formula = cigs ~ log(income) + log(cigpric) + educ + age + 
    I(age^2) + restaurn, weights = 1/hhat)

Weighted Residuals:
    Min      1Q  Median      3Q     Max 
-1.9036 -0.9532 -0.8099  0.8415  9.8556 

Coefficients:
               Estimate Std. Error t value Pr(>|t|)    
(Intercept)   5.6354627 17.8031394   0.317 0.751673    
log(income)   1.2952393  0.4370117   2.964 0.003128 ** 
log(cigpric) -2.9403117  4.4601446  -0.659 0.509931    
educ         -0.4634464  0.1201587  -3.857 0.000124 ***
age           0.4819480  0.0968082   4.978 7.86e-07 ***
I(age^2)     -0.0056272  0.0009395  -5.990 3.17e-09 ***
restaurn     -3.4610640  0.7955050  -4.351 1.53e-05 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 1.579 on 800 degrees of freedom
Multiple R-squared:  0.1134,	Adjusted R-squared:  0.1068 
F-statistic: 17.06 on 6 and 800 DF,  p-value: < 2.2e-16


## C8.10【异方差-稳健标准误；加权最小二乘法（线性概率模型）】
本题利用401KSUBS.RAW。 

（1）利用OLS 估计e401k 的一个线性概率模型，解释变量为inc，inc2，age，age2和male。 求通常的 OLS 标准误和异方差-稳健的标准误。它们有重要的差别吗？ 

（2）在怀特异方差检验的特殊情形中，我们将 OLS 残差的平方对OLS拟合值的二次函数回归 （即$\hat{\mu_i}^2$对$\hat{y_i}$和$\hat{y_i}^2$，i=1，⋯，n），证明$\hat{y_i}$系数的概率极限应该为1，$\hat{y_i}^2$系数的概率极限应该为-1，截距项的概率极限应该为0。
$$\because var(\mu|x)=E(\mu^2|x)-[E(\mu|x)]^2 \quad and\quad 𝐸(𝜇|𝑥)$$
$$\therefore var(\mu|x)=E(\mu^2|x)$$
$$\because var(\mu|x)=var(y|x)$$
$$\therefore E(\mu^2|x)=p(x)(1-p(x))$$
改写为包含误差项的形式
$$\mu^2=p(x)(1-p(x))+\nu$$
写成回归模型的形式
$$\mu^2=\delta_0+\delta_1 p(x)+\delta_2 [p(x)]^2+\nu$$
并且有 $$ \delta_0=0,\delta_1=1,\delta_2=-1$$
$$ and \because \hat{y_i}=p(x)$$
$\therefore$ $\hat{y_i}$系数的概率极限应该为1，$\hat{y_i}^2$系数的概率极限应该为-1，截距项的概率极限应该为0。


（3）对第（1）部分估计的模型求怀特检验，并分析系数估计值是否大致对应于第（2）部分中描述的理论值。 

（4）在验证了第（1）部分的拟合值都介于0 和 1 之后，求这个线性概率模型的加权最小二乘估计。它们与OLS 估计值有重大差别吗？ 

In [65]:
X401ksubs <- read_dta("C:/Users/bradl/SRM/data/401ksubs.dta")
attach(X401ksubs) 

In [66]:
fit1<-lm(e401k~inc+I(inc^2)+age+I(age^2)+male) #拟合上述模型 
summary(fit1) 


Call:
lm(formula = e401k ~ inc + I(inc^2) + age + I(age^2) + male)

Residuals:
    Min      1Q  Median      3Q     Max 
-0.6970 -0.3719 -0.2149  0.4870  0.9155 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept) -5.063e-01  8.110e-02  -6.243 4.48e-10 ***
inc          1.245e-02  5.929e-04  20.993  < 2e-16 ***
I(inc^2)    -6.165e-05  4.732e-06 -13.028  < 2e-16 ***
age          2.651e-02  3.922e-03   6.758 1.49e-11 ***
I(age^2)    -3.053e-04  4.501e-05  -6.782 1.26e-11 ***
male        -3.533e-03  1.208e-02  -0.292     0.77    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 0.4648 on 9269 degrees of freedom
Multiple R-squared:  0.09428,	Adjusted R-squared:  0.09379 
F-statistic:   193 on 5 and 9269 DF,  p-value: < 2.2e-16


In [67]:
coeftest(fit1, vcov=hccm(fit1,type = "hc0")) 


t test of coefficients:

               Estimate  Std. Error  t value  Pr(>|t|)    
(Intercept) -5.0629e-01  7.8529e-02  -6.4472 1.196e-10 ***
inc          1.2446e-02  6.0011e-04  20.7404 < 2.2e-16 ***
I(inc^2)    -6.1649e-05  5.0025e-06 -12.3235 < 2.2e-16 ***
age          2.6506e-02  3.8222e-03   6.9347 4.342e-12 ***
I(age^2)    -3.0527e-04  4.3739e-05  -6.9794 3.169e-12 ***
male        -3.5328e-03  1.2049e-02  -0.2932    0.7694    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1


In [68]:
a<-residuals(fit1) #将fit1 的残差保存在a 中 
b<-fitted(fit1) #将fit1 的拟合值保存在b 中 
fit2<-lm(I(a^2)~b+I(b^2)) #手动计算怀特异方差检验的特例拟合 
summary(fit2) 


Call:
lm(formula = I(a^2) ~ b + I(b^2))

Residuals:
     Min       1Q   Median       3Q      Max 
-0.13158 -0.11178 -0.07017  0.06353  0.76870 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept) -0.009033   0.010915  -0.828    0.408    
b            1.009682   0.057717  17.494   <2e-16 ***
I(b^2)      -0.970286   0.069728 -13.915   <2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 0.154 on 9272 degrees of freedom
Multiple R-squared:  0.06274,	Adjusted R-squared:  0.06254 
F-statistic: 310.3 on 2 and 9272 DF,  p-value: < 2.2e-16


In [71]:
range(fitted(fit1)) #求拟合值的值域，结果为[0.02991717, 0.69718990]，介于0 和1 之间。 
hhat<-b*(1-b) 
fit3<-lm(e401k~inc+I(inc^2)+age+I(age^2)+male, weight=1/hhat) #利用权数1求加权最小二乘估计值
summary(fit3) 
detach(X401ksubs)

[1] 0.02991717 0.69718990


Call:
lm(formula = e401k ~ inc + I(inc^2) + age + I(age^2) + male, 
    weights = 1/hhat)

Weighted Residuals:
    Min      1Q  Median      3Q     Max 
-1.5171 -0.7642 -0.5160  0.9763  3.2889 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept) -4.880e-01  7.559e-02  -6.456 1.13e-10 ***
inc          1.255e-02  5.343e-04  23.491  < 2e-16 ***
I(inc^2)    -6.210e-05  4.182e-06 -14.848  < 2e-16 ***
age          2.550e-02  3.710e-03   6.873 6.71e-12 ***
I(age^2)    -2.950e-04  4.248e-05  -6.944 4.07e-12 ***
male        -5.530e-03  1.171e-02  -0.472    0.637    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 0.9985 on 9269 degrees of freedom
Multiple R-squared:  0.1079,	Adjusted R-squared:  0.1074 
F-statistic: 224.2 on 5 and 9269 DF,  p-value: < 2.2e-16


## C8.11【异方差-稳健的 F 检验；可行 GLS】
本题利用数据集401KSUBS.RAW，仅考虑无子女的已婚夫妇（marr=1,fsize=2）。 

（1）用OLS 估计方程 $$nettfa=\beta_0+\beta_1inc+\beta_2(inc−10)^2+\beta_3age+\beta_4(age−25)^2+\beta_5e401k+\mu$$ 二次项通过减去 inc 和age 的最小值而加以修正，所以$\beta_1$是inc 在inc=10 时对nettfa 的偏效应，$\beta_3$是age在age=25 时对nettfa的偏效应。报告通常的标准误和异方差-稳健的标准误。 

（2）利用一个异方差-稳健的检验，检验inc和age的联合显著性。 

（3）利用 WLS 估计第（1）部分中的模型，其中$\hat{h_i}$得自方程（8.33）。求通常的WLS标准误和对异方差函数形式误设保持稳健的标准误。就检验inc和age的联合显著性，比较通常的WLS检验和稳健的WLS检验。 

（4）比较$\beta_{e401k}$的OLS估计值和WLS估计值。你认为出现了什么样的情况？ 

In [73]:
a<-subset(X401ksubs, marr==1 & fsize==2) 
attach(a) 

In [74]:
fit1<-lm(nettfa~inc+I((inc-10)^2)+age+ I((age-25)^2)+e401k) #拟合上述模型 
summary(fit1) 


Call:
lm(formula = nettfa ~ inc + I((inc - 10)^2) + age + I((age - 
    25)^2) + e401k)

Residuals:
    Min      1Q  Median      3Q     Max 
-521.68  -26.31   -6.21   12.47 1455.23 

Coefficients:
                  Estimate Std. Error t value Pr(>|t|)    
(Intercept)     -15.904224  22.696739  -0.701 0.483583    
inc              -0.325537   0.215127  -1.513 0.130433    
I((inc - 10)^2)   0.013380   0.001659   8.066 1.48e-15 ***
age               0.226082   0.703940   0.321 0.748129    
I((age - 25)^2)   0.035585   0.017716   2.009 0.044753 *  
e401k            15.003104   4.358100   3.443 0.000592 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 79.82 on 1488 degrees of freedom
Multiple R-squared:  0.2405,	Adjusted R-squared:  0.2379 
F-statistic: 94.23 on 5 and 1488 DF,  p-value: < 2.2e-16


In [75]:
coeftest(fit1, vcov=hccm(fit1,type = "hc0")) 


t test of coefficients:

                   Estimate  Std. Error t value  Pr(>|t|)    
(Intercept)     -15.9042242  26.1461781 -0.6083 0.5430939    
inc              -0.3255367   0.5358650 -0.6075 0.5436135    
I((inc - 10)^2)   0.0133798   0.0056637  2.3624 0.0182861 *  
age               0.2260824   0.6376588  0.3546 0.7229764    
I((age - 25)^2)   0.0355850   0.0184138  1.9325 0.0534851 .  
e401k            15.0031039   4.2375809  3.5405 0.0004117 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1


In [76]:
myH0<-c("inc","age") #用联合检验所涉及的变量组成向量，构成H0。 
linearHypothesis(fit1, myH0, vcov=hccm(fit1,type="hc0")) 

Res.Df,Df,F,Pr(>F)
<dbl>,<dbl>,<dbl>,<dbl>
1490,NA,NA,NA
1488,2,0.3323014,0.7173243


In [78]:
b<-residuals(fit1) #将fit1 的残差保存在b 中 
fit2<-lm(I(log(b^2))~ inc+I((inc-10)^2)+age+ I((age-25)^2)+e401k) #做方程8.32 中的回归 
c<-fitted(fit2) #得到fit2 中的拟合值 
hhat<-exp(c) #得到WLS 估计中所用的
fit3<-lm(nettfa~inc+I((inc-10)^2)+age+ I((age-25)^2)+e401k, weight=1/hhat) #重新计算加权最小二 乘得到的方程8.36 
summary(fit3) 


Call:
lm(formula = nettfa ~ inc + I((inc - 10)^2) + age + I((age - 
    25)^2) + e401k, weights = 1/hhat)

Weighted Residuals:
    Min      1Q  Median      3Q     Max 
-17.878  -1.259  -0.478   0.646  43.623 

Coefficients:
                  Estimate Std. Error t value Pr(>|t|)   
(Intercept)     -31.225211  10.505678  -2.972   0.0030 **
inc               0.338673   0.155476   2.178   0.0295 * 
I((inc - 10)^2)   0.003790   0.001987   1.908   0.0566 . 
age               0.562282   0.329327   1.707   0.0880 . 
I((age - 25)^2)   0.016522   0.009709   1.702   0.0890 . 
e401k             4.478437   2.072971   2.160   0.0309 * 
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 2.687 on 1488 degrees of freedom
Multiple R-squared:  0.1622,	Adjusted R-squared:  0.1594 
F-statistic: 57.61 on 5 and 1488 DF,  p-value: < 2.2e-16


In [79]:
coeftest(fit3, vcov=hccm(fit3,type = "hc0")) 


t test of coefficients:

                   Estimate  Std. Error t value  Pr(>|t|)    
(Intercept)     -31.2252114   8.8325864 -3.5352 0.0004199 ***
inc               0.3386731   0.1405785  2.4091 0.0161110 *  
I((inc - 10)^2)   0.0037897   0.0018182  2.0843 0.0373003 *  
age               0.5622822   0.2616368  2.1491 0.0317878 *  
I((age - 25)^2)   0.0165221   0.0085982  1.9216 0.0548505 .  
e401k             4.4784369   1.9032909  2.3530 0.0187524 *  
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1


In [80]:
myH0<-c("inc","age") #用联合检验所涉及的变量组成向量，构成H0。 
linearHypothesis(fit3, myH0, vcov=hccm(fit3,type="hc0")) 

Res.Df,Df,F,Pr(>F)
<dbl>,<dbl>,<dbl>,<dbl>
1490,NA,NA,NA
1488,2,4.72027,0.009046662


$\beta_{e401k}$在OLS 中的估计值是15.00304，在WLS 中的估计值是4.478437，差异较大。这标志着其违反了高斯-马尔可夫假定，特别是对误差的零条件均值假定。$\mu$可能和e401k之间存在相关关系。 

## C8.12【异方差-稳健标准误；特殊怀特检验；加权最小二乘法（WLS）】 
本题利用数据集MEAP00_01.RAW 中的数据。 

（1）用OLS 估计方程 $$math4=\beta_0+\beta_1lunch+\beta_2\log{enroll}+\beta_3\log{exppp}+\mu$$ 并求出通常的标准误和完全稳健的标准误。两者相比如何？

（2）对异方差性应用怀特检验的特殊形式，F 检验的值是多少？你得到什么结论？ 

（3）从$log{(\hat{\mu_i}^2)}$ 对$\widehat{math4_l}$和$\widehat{math4_l^2}$的回归中求拟合值$\hat{g_l}$，其中$\widehat{math4_l}$是OLS 拟合值，而$\hat{\mu_i}$是OLS 残差。令$\hat{h_i}=\exp{\hat{g_i}}$。利用$\hat{h_i}$求WLS估计值。它与OLS系数存在巨大差异吗？ 

（4）求容许方差函数被误设的WLS 标准误。它与通常的WLS 标准误有很大的不同吗？ 

（5）为了估计支出对 math4 的影响，OLS 与WLS 哪一个看起来更准确？ 

In [81]:
attach(meap00_01) 

In [82]:
fit1<-lm(math4~lunch+log(enroll)+log(exppp)) #拟合上述模型 
summary(fit1) 


Call:
lm(formula = math4 ~ lunch + log(enroll) + log(exppp))

Residuals:
    Min      1Q  Median      3Q     Max 
-56.326  -8.758   0.914   9.164  51.416 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept) 91.93240   19.96170   4.605 4.42e-06 ***
lunch       -0.44874    0.01464 -30.648  < 2e-16 ***
log(enroll) -5.39915    0.94041  -5.741 1.11e-08 ***
log(exppp)   3.52475    2.09785   1.680   0.0931 .  
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 15.3 on 1688 degrees of freedom
Multiple R-squared:  0.3729,	Adjusted R-squared:  0.3718 
F-statistic: 334.6 on 3 and 1688 DF,  p-value: < 2.2e-16


In [83]:
coeftest(fit1, vcov=hccm(fit1,type = "hc0")) 


t test of coefficients:

             Estimate Std. Error  t value  Pr(>|t|)    
(Intercept) 91.932404  23.059797   3.9867 6.986e-05 ***
lunch       -0.448743   0.016565 -27.0903 < 2.2e-16 ***
log(enroll) -5.399152   1.129837  -4.7787 1.917e-06 ***
log(exppp)   3.524751   2.350953   1.4993     0.134    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1


异方差-稳健的标准误比普通OLS 标准误大。log(exppp)系数的t值在采用异方差-稳健标准误之后不再显著。

In [84]:
bptest(fit1,~fitted(fit1)+I(fitted(fit1)^2)) 


	studentized Breusch-Pagan test

data:  fit1
BP = 229.78, df = 2, p-value < 2.2e-16


In [85]:
a<-residuals(fit1) #将fit1 的残差保存在a 中 
b<-fitted(fit1) #将fit1 的拟合值保存在b 中 
fit2<-lm(I(log(a^2))~b+I(b^2)) #拟合上述模型 
c<-fitted(fit2) 
hhat<-exp(c) 
fit3<-lm(math4~lunch+log(enroll)+log(exppp), weight=1/hhat) 
summary(fit3) 


Call:
lm(formula = math4 ~ lunch + log(enroll) + log(exppp), weights = 1/hhat)

Weighted Residuals:
     Min       1Q   Median       3Q      Max 
-10.7122  -1.1914   0.1403   1.3741   3.9151 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept) 50.47815   16.51032   3.057 0.002268 ** 
lunch       -0.44859    0.01461 -30.697  < 2e-16 ***
log(enroll) -2.64728    0.83594  -3.167 0.001569 ** 
log(exppp)   6.47420    1.68588   3.840 0.000127 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 1.9 on 1688 degrees of freedom
Multiple R-squared:   0.36,	Adjusted R-squared:  0.3588 
F-statistic: 316.5 on 3 and 1688 DF,  p-value: < 2.2e-16


和OLS 的相比，lunch 的系数及t 值没有明显变化，但其他三个变量的系数有较大变化。特别 是log(exppp)的系数明显更大，而t 值明显变小了，但仍在 10%水平上显著。 

In [86]:
coeftest(fit3, vcov=hccm(fit3,type = "hc0")) 


t test of coefficients:

             Estimate Std. Error  t value  Pr(>|t|)    
(Intercept) 50.478152  18.902796   2.6704 0.0076487 ** 
lunch       -0.448592   0.014237 -31.5084 < 2.2e-16 ***
log(enroll) -2.647282   1.053361  -2.5132 0.0120574 *  
log(exppp)   6.474200   1.810724   3.5755 0.0003594 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1


在异方差-稳健的标准误中，log(enroll)的标准误明显变大了，log(exppp)的标准误也有所增加， lunch 的标准误基本不变。 